# Chapter 1: EDA & Forecasting Pipeline

A step-by-step guide to building a production-ready time series forecasting pipeline.

## Learning Steps

1. **Configuration + Secrets** - Load API key from environment
2. **Ingest from EIA API** - Paginated requests with stable sort
3. **Prepare / Normalize Time** - Standardize to UTC
4. **Validate Time Series Integrity** - Check duplicates, missing hours
5. **Split for Forecasting** - Time-based train/test split
6. **Build Models** - Statistical forecasting with statsforecast
7. **Evaluate** - Metrics and visualization

---
## Step 0: Setup

Add the src directory to the path so we can import our modules.

In [2]:
import sys
from pathlib import Path

# Add src to path
src_path = Path.cwd().parent / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Added to path: {src_path}")

Added to path: c:\docker_projects\atsaf\src


---
## Step 1: Configuration + Secrets

**Packages:** `os`, `python-dotenv`

**What we're doing:**
- Keep `EIA_API_KEY` in `.env` file (local) or environment variable (prod)
- Use a `Settings` object so every run uses the same config

In [3]:
from chapter1 import load_settings

# Load configuration from .env file
settings = load_settings(
    respondent="US48",       # Lower 48 states
    fueltype="NG",           # Natural Gas
    start_date="2024-01-01",
    end_date="2024-12-31",
)

# Show config (without exposing API key)
print(f"Respondent: {settings.respondent}")
print(f"Fuel Type: {settings.fueltype}")
print(f"Date Range: {settings.start_date} to {settings.end_date}")
print(f"API Key: {'*' * 8}...{settings.api_key[-4:]}")

Respondent: US48
Fuel Type: NG
Date Range: 2024-01-01 to 2024-12-31
API Key: ********...8Xk7


---
## Step 2: Ingest from EIA API (Pagination + Stable Sort)

**Packages:** `requests`, `urllib3`, `pandas`

**What we're doing:**
- Use EIA API v2 `length` and `offset` for pagination
- Add stable sorting (`sort[0][column]=period`, `sort[0][direction]=asc`) so pages don't shuffle
- Use `requests.Session()` with retries + timeouts

**Pattern:**
1. `offset=0`
2. GET with params including `length`, `offset`, and `sort`
3. Append records
4. Stop when returned records < `length`

In [4]:
from chapter1 import pull_data_paged

# Pull raw data with pagination
df_raw = pull_data_paged(settings)

# Inspect raw data
print(f"\nShape: {df_raw.shape}")
print(f"Columns: {list(df_raw.columns)}")
df_raw.head()

Pulling data: 2024-01-01 to 2024-12-31
  Respondent: US48, Fuel: NG
  Page 1: 5000 records
  Page 2: 3761 records
  Total: 8761 records

Shape: (8761, 7)
Columns: ['period', 'respondent', 'respondent-name', 'fueltype', 'type-name', 'value', 'value-units']


,period,respondent,respondent-name,fueltype,type-name,value,value-units
0,2024-01-01T00,US48,United States Lower 48,NG,Natural Gas,194720,megawatthours
1,2024-01-01T01,US48,United States Lower 48,NG,Natural Gas,193932,megawatthours
2,2024-01-01T02,US48,United States Lower 48,NG,Natural Gas,190651,megawatthours
3,2024-01-01T03,US48,United States Lower 48,NG,Natural Gas,187166,megawatthours
4,2024-01-01T04,US48,United States Lower 48,NG,Natural Gas,184852,megawatthours


---
## Step 3: Prepare / Normalize Time

**Packages:** `pandas`

**What we're doing:**
- `pd.to_datetime(period, utc=True)` to standardize to UTC
- Convert value to numeric
- Keep a `tz_source` field for debugging

In [5]:
from chapter1 import normalize_time

# Normalize to UTC and clean types
df_clean = normalize_time(df_raw)

# Inspect cleaned data
print(f"\nDtypes:")
print(df_clean.dtypes)
print(f"\nFirst few rows:")
df_clean.head()

Normalized: 8761 rows, 2024-01-01 00:00:00+00:00 to 2024-12-31 00:00:00+00:00

Dtypes:
period        datetime64[ns, UTC]
value                       int64
respondent                 object
fueltype                   object
tz_source                  object
dtype: object

First few rows:


,period,value,respondent,fueltype,tz_source
0,2024-01-01 00:00:00+00:00,194720,US48,NG,UTC
1,2024-01-01 01:00:00+00:00,193932,US48,NG,UTC
2,2024-01-01 02:00:00+00:00,190651,US48,NG,UTC
3,2024-01-01 03:00:00+00:00,187166,US48,NG,UTC
4,2024-01-01 04:00:00+00:00,184852,US48,NG,UTC


---
## Step 4: Validate Time Series Integrity

**Packages:** `pandas`

**Hard gates:**
- Uniqueness: no duplicates on `[unique_id, ds]`
- Frequency: expected hourly index vs observed (missing/repeated hours)
- Monotonic: increasing time
- Value sanity: min/max bounds, missing rate

In [6]:
from chapter1 import prepare_for_forecasting, validate_time_index, print_validation_report

# Convert to forecasting format
df_forecast = prepare_for_forecasting(df_clean, unique_id="NG_US48")

# Validate time series integrity
result = validate_time_index(df_forecast)
print_validation_report(result)

Forecast format: 8761 rows, unique_id=NG_US48

=== Validation Report: PASS ===
Rows: 8761
Duplicates: 0
Missing hours: 0
Null values: 0
Value range: 104793 to 348914
Monotonic: True


---
## Step 5: Save Artifacts

**Artifacts to write every run:**
- `raw.parquet` - Raw API response
- `clean.parquet` - Cleaned data
- `metadata.json` - Params, pull timestamp, row counts

In [7]:
import json
from datetime import datetime
from pathlib import Path

# Create data directory
data_dir = Path.cwd().parent / "data"
data_dir.mkdir(exist_ok=True)

# Save raw and clean data
df_raw.to_parquet(data_dir / "raw.parquet", index=False)
df_clean.to_parquet(data_dir / "clean.parquet", index=False)

# Save metadata
metadata = {
    "pull_timestamp": datetime.utcnow().isoformat(),
    "respondent": settings.respondent,
    "fueltype": settings.fueltype,
    "start_date": settings.start_date,
    "end_date": settings.end_date,
    "raw_rows": len(df_raw),
    "clean_rows": len(df_clean),
    "missing_hours": result.n_missing_hours,
    "validation_passed": result.is_valid,
}

with open(data_dir / "metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print(f"Saved artifacts to {data_dir}")
print(json.dumps(metadata, indent=2))

Saved artifacts to c:\docker_projects\atsaf\data
{
  "pull_timestamp": "2026-01-08T19:05:31.701421",
  "respondent": "US48",
  "fueltype": "NG",
  "start_date": "2024-01-01",
  "end_date": "2024-12-31",
  "raw_rows": 8761,
  "clean_rows": 8761,
  "missing_hours": 0,
  "validation_passed": true
}


C:\Users\ghadf\AppData\Local\Temp\ipykernel_244192\470448194.py:15: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



---
## Step 6: Train/Test Split (Time-Based)

**Key principle:** Always split by time, not random.

We'll leave the last 72 hours (3 days) for testing.

In [8]:
from datetime import timedelta

# Time-based split
test_hours = 72
split_point = df_forecast["ds"].max() - timedelta(hours=test_hours)

train_df = df_forecast[df_forecast["ds"] <= split_point].reset_index(drop=True)
test_df = df_forecast[df_forecast["ds"] > split_point].reset_index(drop=True)

print(f"Split point: {split_point}")
print(f"Train: {len(train_df)} rows ({train_df['ds'].min()} to {train_df['ds'].max()})")
print(f"Test:  {len(test_df)} rows ({test_df['ds'].min()} to {test_df['ds'].max()})")

Split point: 2024-12-28 00:00:00+00:00
Train: 8689 rows (2024-01-01 00:00:00+00:00 to 2024-12-28 00:00:00+00:00)
Test:  72 rows (2024-12-28 01:00:00+00:00 to 2024-12-31 00:00:00+00:00)


---
## Step 7: Build Forecasting Models

**Packages:** `statsforecast`

Models we'll use:
- AutoARIMA - Auto-tuned ARIMA
- SeasonalNaive - Baseline (same value from 24 hours ago)
- MSTL - Multi-seasonal decomposition

In [9]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, SeasonalNaive, MSTL

# Define models
models = [
    SeasonalNaive(season_length=24),
    AutoARIMA(season_length=24),
    MSTL(season_length=[24, 24 * 7], trend_forecaster=AutoARIMA()),
]

# Create forecaster
sf = StatsForecast(
    models=models,
    freq="h",
    n_jobs=-1,
)

print(f"Models: {[type(m).__name__ for m in models]}")

Models: ['SeasonalNaive', 'AutoARIMA', 'MSTL']


---
## Step 8: Fit and Forecast

In [10]:
# Fit models on training data
sf.fit(train_df)

# Forecast for test period
horizon = len(test_df)
forecast_df = sf.predict(h=horizon, level=[95])

print(f"Forecast shape: {forecast_df.shape}")
forecast_df.head()

Forecast shape: (72, 11)


,unique_id,ds,SeasonalNaive,SeasonalNaive-lo-95,SeasonalNaive-hi-95,AutoARIMA,AutoARIMA-lo-95,AutoARIMA-hi-95,MSTL,MSTL-lo-95,MSTL-hi-95
0,NG_US48,2024-12-28 01:00:00+00:00,200352.0,168175.640625,232528.359375,193758.093750,187927.703125,199588.468750,191240.546875,187436.359375,195044.734375
1,NG_US48,2024-12-28 02:00:00+00:00,197825.0,165648.640625,230001.359375,190042.828125,180405.625000,199680.031250,187924.453125,181883.265625,193965.640625
2,NG_US48,2024-12-28 03:00:00+00:00,192065.0,159888.640625,224241.359375,185334.468750,172260.906250,198408.046875,184812.203125,176719.703125,192904.703125
3,NG_US48,2024-12-28 04:00:00+00:00,185103.0,152926.640625,217279.359375,179774.671875,163830.781250,195718.546875,180691.578125,170793.046875,190590.109375
4,NG_US48,2024-12-28 05:00:00+00:00,176019.0,143842.640625,208195.359375,173870.000000,155506.750000,192233.234375,176013.859375,164518.765625,187508.968750


---
## Step 9: Evaluate Forecast Performance

**Metrics:**
- RMSE - Root Mean Squared Error
- MAPE - Mean Absolute Percentage Error
- Coverage - % of actuals within 95% prediction interval

In [11]:
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

def coverage(y_true, lo, hi):
    return 100 * np.mean((y_true >= lo) & (y_true <= hi))

# Merge actuals with forecasts
eval_df = test_df.merge(forecast_df.reset_index(), on=["unique_id", "ds"])

# Calculate metrics for each model
model_names = ["SeasonalNaive", "AutoARIMA", "MSTL"]
results = []

for model in model_names:
    y_true = eval_df["y"].values
    y_pred = eval_df[model].values
    
    lo_col = f"{model}-lo-95"
    hi_col = f"{model}-hi-95"
    
    cov = 0
    if lo_col in eval_df.columns and hi_col in eval_df.columns:
        cov = coverage(y_true, eval_df[lo_col].values, eval_df[hi_col].values)
    
    results.append({
        "model": model,
        "rmse": rmse(y_true, y_pred),
        "mape": mape(y_true, y_pred),
        "coverage_95": cov,
    })

import pandas as pd
metrics_df = pd.DataFrame(results).sort_values("rmse")
print("Model Performance:")
metrics_df

Model Performance:


,model,rmse,mape,coverage_95
2,MSTL,18072.908292,10.324333,88.888889
1,AutoARIMA,22700.459886,14.339329,100.000000
0,SeasonalNaive,22754.494320,14.598983,100.000000


---
## Step 10: Visualize Forecast vs Actuals

In [12]:
import plotly.graph_objects as go

# Get best model (lowest RMSE)
best_model = metrics_df.iloc[0]["model"]
print(f"Best model: {best_model}")

fig = go.Figure()

# Actual values
fig.add_trace(go.Scatter(
    x=eval_df["ds"],
    y=eval_df["y"],
    mode="lines",
    name="Actual",
    line=dict(color="blue", width=2),
))

# Forecast
fig.add_trace(go.Scatter(
    x=eval_df["ds"],
    y=eval_df[best_model],
    mode="lines",
    name=best_model,
    line=dict(color="red", width=2, dash="dash"),
))

# 95% Prediction Interval
lo_col = f"{best_model}-lo-95"
hi_col = f"{best_model}-hi-95"

if lo_col in eval_df.columns:
    fig.add_trace(go.Scatter(
        x=list(eval_df["ds"]) + list(eval_df["ds"][::-1]),
        y=list(eval_df[hi_col]) + list(eval_df[lo_col][::-1]),
        fill="toself",
        fillcolor="rgba(255, 0, 0, 0.2)",
        line=dict(color="rgba(255, 0, 0, 0)"),
        name="95% PI",
    ))

fig.update_layout(
    title=f"EIA Natural Gas Generation: {best_model} Forecast vs Actual",
    xaxis_title="Date",
    yaxis_title="Generation (MWh)",
    height=400,
    template="plotly_white",
)

fig.show()

Best model: MSTL


---
# Chapter 2: Experimentation (Backtesting + MLflow)

Now we move from single train/test split to proper backtesting with rolling origin cross-validation.

---
## Step 11: Define Experiment Config

**What we're doing:**
- Create an experiment config with:
  - horizon `h`
  - `n_windows`, `step_size`
  - model list
  - metrics list

In [13]:
from dataclasses import dataclass
from typing import List

@dataclass
class ExperimentConfig:
    """Configuration for backtesting experiment"""
    name: str
    horizon: int           # Forecast horizon (hours)
    n_windows: int         # Number of CV windows
    step_size: int         # Hours between windows
    models: List[str]      # Model names
    metrics: List[str]     # Metrics to compute

# Define experiment
experiment = ExperimentConfig(
    name="ng_us48_hourly",
    horizon=24,
    n_windows=5,
    step_size=24 * 7,  # Weekly steps
    models=["SeasonalNaive", "AutoARIMA", "MSTL"],
    metrics=["rmse", "mape", "coverage"],
)

print(f"Experiment: {experiment.name}")
print(f"Horizon: {experiment.horizon}h")
print(f"Windows: {experiment.n_windows} with {experiment.step_size}h step")

Experiment: ng_us48_hourly
Horizon: 24h
Windows: 5 with 168h step


---
## Step 12: Backtesting (Rolling Origin Cross-Validation)

**Packages:** `statsforecast`

**What we're doing:**
- Use `StatsForecast.cross_validation(...)` for rolling origin CV
- Output unified "long" backtest table with cutoff dates

In [ ]:
# Run cross-validation
cv_df = sf.cross_validation(
    df=df_forecast,
    h=experiment.horizon,
    step_size=experiment.step_size,
    n_windows=experiment.n_windows,
    level=[95],
)

print(f"CV results shape: {cv_df.shape}")
print(f"Cutoff dates: {cv_df['cutoff'].unique()}")
cv_df.head()

---
## Step 13: Compute Metrics Per Window

**What we're doing:**
- Calculate RMSE, MAPE, Coverage per model per window
- Aggregate: mean across windows + stability (std)

In [ ]:
# Compute metrics per cutoff per model
cv_metrics = []

for cutoff in cv_df["cutoff"].unique():
    window_df = cv_df[cv_df["cutoff"] == cutoff]
    y_true = window_df["y"].values
    
    for model in model_names:
        y_pred = window_df[model].values
        
        lo_col = f"{model}-lo-95"
        hi_col = f"{model}-hi-95"
        
        cov = 0
        if lo_col in window_df.columns:
            cov = coverage(y_true, window_df[lo_col].values, window_df[hi_col].values)
        
        cv_metrics.append({
            "cutoff": cutoff,
            "model": model,
            "rmse": rmse(y_true, y_pred),
            "mape": mape(y_true, y_pred),
            "coverage": cov,
        })

cv_metrics_df = pd.DataFrame(cv_metrics)
print("Metrics per window:")
cv_metrics_df.head(10)

---
## Step 14: Leaderboard

Aggregate metrics across all windows to create a model leaderboard.

In [ ]:
# Aggregate across windows
leaderboard = cv_metrics_df.groupby("model").agg({
    "rmse": ["mean", "std"],
    "mape": ["mean", "std"],
    "coverage": "mean",
}).round(2)

# Flatten column names
leaderboard.columns = ["_".join(col).strip() for col in leaderboard.columns.values]
leaderboard = leaderboard.sort_values("rmse_mean").reset_index()

print("=== LEADERBOARD ===")
leaderboard

---
## Step 15: MLflow Logging

**Packages:** `mlflow`

**What we're doing:**
- Log experiment config as params
- Log metrics for each model
- Log leaderboard and CV results as artifacts

In [ ]:
import mlflow

# Set experiment
mlflow.set_experiment(experiment.name)

with mlflow.start_run(run_name=f"backtest_{experiment.n_windows}windows"):
    # Log config
    mlflow.log_param("horizon", experiment.horizon)
    mlflow.log_param("n_windows", experiment.n_windows)
    mlflow.log_param("step_size", experiment.step_size)
    mlflow.log_param("data_start", settings.start_date)
    mlflow.log_param("data_end", settings.end_date)
    
    # Log metrics for best model
    best = leaderboard.iloc[0]
    mlflow.log_metric("best_rmse", best["rmse_mean"])
    mlflow.log_metric("best_mape", best["mape_mean"])
    mlflow.log_metric("best_coverage", best["coverage_mean"])
    mlflow.log_param("best_model", best["model"])
    
    # Save artifacts
    artifacts_dir = Path.cwd().parent / "artifacts"
    artifacts_dir.mkdir(exist_ok=True)
    
    leaderboard.to_parquet(artifacts_dir / "leaderboard.parquet", index=False)
    cv_metrics_df.to_parquet(artifacts_dir / "cv_metrics.parquet", index=False)
    
    mlflow.log_artifacts(str(artifacts_dir))
    
    run_id = mlflow.active_run().info.run_id
    print(f"MLflow run: {run_id}")
    print(f"Best model: {best['model']} (RMSE: {best['rmse_mean']:.2f})")

---
## Step 16: Visualize CV Results

In [ ]:
import plotly.express as px

# RMSE across windows
fig = px.line(
    cv_metrics_df,
    x="cutoff",
    y="rmse",
    color="model",
    markers=True,
    title="RMSE Across CV Windows",
)
fig.update_layout(height=400, template="plotly_white")
fig.show()

---
## Summary

### Chapter 1 - Data Pipeline
- **Step 1:** Loaded config from `.env`
- **Step 2:** Pulled data with pagination + stable sort
- **Step 3:** Normalized time to UTC
- **Step 4:** Validated time series integrity (duplicates, missing hours)
- **Step 5:** Saved artifacts (raw, clean, metadata)
- **Step 6:** Time-based train/test split

### Chapter 2 - Experimentation
- **Step 11:** Defined experiment config
- **Step 12:** Rolling origin cross-validation
- **Step 13:** Metrics per window
- **Step 14:** Leaderboard aggregation
- **Step 15:** MLflow logging

### What's Next (Chapters 3-4)
- **Automation:** Airflow DAGs for ETL + training
- **Monitoring:** Drift detection, freshness checks, dashboards